In [4]:
import unittest
import sys
sys.path.append('../../../../')
from pathlib import Path

import torch
import onnx

from torchok.constructor import BACKBONES


In [75]:
class TestOnnx(unittest.TestCase):
    def __init__(self, testName, backbone_name):
        super().__init__(testName)
        self.backbone_name = backbone_name

    def test_onnx(self):
        self._onnx_model = Path(f'{self.backbone_name}.onnx')
        self._input = torch.ones(1, 3, 224, 224)
        #self._input = torch.ones(1, 3, 384, 384)
        #self._input = torch.ones(1, 3, 256, 256)
        self.backbone = BACKBONES.get(self.backbone_name)(pretrained=False, in_chans=3)
        torch.onnx.export(self.backbone,
                          self._input,
                          self._onnx_model,
                          opset_version=12)
        model = onnx.load(self._onnx_model)
        onnx.checker.check_model(model)
        self._onnx_model.unlink()

In [ ]:
for backbone_name in BACKBONES.entrypoints.keys():
    print(backbone_name)
    suite = unittest.TestSuite()
    suite.addTest(TestOnnx('test_onnx', backbone_name))
    unittest.TextTestRunner(verbosity=2).run(suite)